# Resume Optimizer

This interactive tool helps you optimize your resume for specific job applications by:
1. **Uploading your current resume** (PDF, DOCX, or MD)
2. Selecting a job type with pre-configured keywords
3. Choosing a template optimized for your target role
4. Adding missing keywords to ensure ATS compatibility
5. Generating a professionally formatted PDF resume

## Instructions:

1. Complete the form below:
   - **Upload your resume:** 
     - **⚠️ Warning:** Parsing structure from PDF files can be unreliable due to varying formats. For the best interactive editing experience (sections, bullets), please upload a **DOCX or Markdown (.md)** file if possible.
     - Alternatively, consider using a base template and filling it out with the editor (future enhancement).
   - Enter your full name (first and last)
   - Select a job type (e.g., Data Engineer)
   - Choose a template variant (e.g., general, company-specific)
   - Set whether to upload the final PDF to Google Drive

2. Click the "Generate Resume" button.
   - The first click parses your resume and shows an editor (results may vary for PDFs).
   - Make edits in the editor below.
   - **Optional:** Click 'Suggest' next to a bullet point for AI improvement ideas.
   - **Optional:** If a suggestion is generated, click 'Apply Suggestion' to update the bullet point.
   - **Optional:** Edit the comma-separated list in the 'Section Order' box to reorder sections.
   - **Optional:** Enter a name and click 'Save as Template' to save your current edits as a new template for this job type.
   - Click "Generate Resume" again to create the final output based on your edits and desired order. A comparison of keyword coverage will be shown.

3. Your files will be saved in the `data/output` directory.

In [ ]:
import os
import sys
import pickle
import re # Needed for template name sanitization
import tqdm.notebook as tqdm
import glob
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML # Added HTML for display
from pathlib import Path
import pandas as pd # Ensure pandas is imported
import shutil # For saving uploaded file
from functools import partial # For button callbacks
from dotenv import load_dotenv # To load .env file
import google.generativeai as genai # Import Google AI library

# Load environment variables from .env file
load_dotenv()

# Add scripts directory to path
sys.path.append('scripts')

# Import required modules (Updated)
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
# Updated import: added save_structure_as_template
from scripts.resume_generator import load_keywords_for_job, parse_resume, parse_structured_resume, calculate_keyword_coverage, analyze_keyword_distribution, generate_resume_from_structure, save_structure_as_template, generate_resume, generate_pdf, upload_to_drive

# --- Setup Directories ---
input_dir = "data/input"
output_dir = "data/output"
job_types_base_dir = "job_types"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(job_types_base_dir, exist_ok=True)

# --- Function to Load Job Types and Templates --- 
def load_job_types_and_templates():
    """Loads available job types and their templates from the filesystem."""
    loaded_job_types = []
    loaded_templates = {}
    if os.path.exists(job_types_base_dir):
        try:
            loaded_job_types = sorted([name for name in os.listdir(job_types_base_dir) if os.path.isdir(os.path.join(job_types_base_dir, name))])
            for job_type in loaded_job_types:
                templates_dir = os.path.join(job_types_base_dir, job_type, 'templates')
                if os.path.exists(templates_dir):
                    templates = sorted([os.path.splitext(name)[0] for name in os.listdir(templates_dir) if name.endswith('.md')])
                    loaded_templates[job_type] = templates
                else:
                     loaded_templates[job_type] = [] # Ensure key exists even if no templates
        except Exception as e:
            print(f"Error loading job types/templates: {e}")
    return loaded_job_types, loaded_templates

# --- Initial Load --- 
job_types, job_types_templates = load_job_types_and_templates()

# --- Create Widgets ---
file_upload = widgets.FileUpload(
    accept='.pdf,.docx,.md',  # Accepted file types
    multiple=False,  # Allow only single file upload
    description='Upload Resume:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

name_input = widgets.Text(
    value='Malachi Dunn',
    description='Full Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

job_type_dropdown = widgets.Dropdown(
    options=job_types,
    value=job_types[0] if job_types else None,
    description='Job Type:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

template_dropdown = widgets.Dropdown(
    options=job_types_templates.get(job_types[0], []) if job_types else [],
    value=job_types_templates.get(job_types[0], [])[0] if job_types and job_types_templates.get(job_types[0], []) else None,
    description='Template:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

upload_dropdown = widgets.Dropdown(
    options=[('Yes', True), ('No', False), ('Skip PDF generation', 'skip')],
    value=True,
    description='Upload to Google Drive:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Widget to display coverage results & distribution
coverage_output = widgets.HTML(value="")

# Placeholder for interactive editing UI
interactive_area = widgets.VBox([])
# Widget for section reordering
section_order_input = widgets.Textarea(
    value='',
    description='Section Order:',
    placeholder='Edit comma-separated section names to reorder',
    layout=widgets.Layout(width='95%', height='50px'),
    style={'description_width': 'initial'},
    disabled=True # Disabled until editor is populated
)

# Widgets for saving template
new_template_name_input = widgets.Text(
    value='',
    description='New Template Name:',
    placeholder='Enter name for new template (e.g., my_custom_swe)',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'},
    disabled=True
)
save_template_button = widgets.Button(
    description='Save as Template',
    button_style='warning',
    icon='save',
    tooltip='Save the current editor content as a new template for this job type',
    disabled=True
)
save_template_output_area = widgets.Output()

# Dictionary to store references to the interactive widgets (Textareas)
interactive_widgets = {}
# Store parsed content globally for coverage check on second click
parsed_resume_content_global = None
# Store structured resume globally for distribution analysis & saving
structured_resume_global = None
# Store original coverage results for comparison
original_coverage_results = {'percent': 0.0, 'found_kws': []}
# Store original section keys for validation
original_section_keys = []
# Store loaded keywords globally for suggestion handler
keywords_df_global = pd.DataFrame()

# Static values
drive_folder_id = "1s6n9eS9d2NNy9lyXgoYdPGOwkyGGujnw"
keywords_db_file = 'keywords_db.csv' # Central keywords DB

# --- Suggestion Handling --- 
suggestion_output_area = widgets.Output()
apply_button_area = widgets.VBox([]) # Separate area for the apply button

def on_apply_suggestion_click(target_widget, suggestion_text, b):
    target_widget.value = suggestion_text
    with suggestion_output_area:
        clear_output(wait=True) 
        display(HTML("<i style='color: green;'>Suggestion applied!</i>"))
    apply_button_area.children = [] 

def on_suggest_click(section_name, item_index, b):
    global interactive_widgets, keywords_df_global, parsed_resume_content_global
    suggestion_output_area.clear_output()
    apply_button_area.children = []
    
    with suggestion_output_area:
        display(HTML("<i>⏳ Generating AI suggestion...</i>"))
        
        api_key = os.getenv("GOOGLE_API_KEY")
        if not api_key:
            clear_output(wait=True)
            display(HTML("<p style='color: red;'>❌ Error: GOOGLE_API_KEY not found. Make sure it's set in your .env file.</p>"))
            return
            
        try:
            genai.configure(api_key=api_key)
        except Exception as e:
             clear_output(wait=True)
             display(HTML(f"<p style='color: red;'>❌ Error configuring Google AI SDK: {e}</p>"))
             return
             
        if section_name in interactive_widgets and item_index < len(interactive_widgets[section_name]):
            bullet_widget = interactive_widgets[section_name][item_index]
            bullet_text = bullet_widget.value
            
            missing_keywords = []
            if parsed_resume_content_global is not None and not keywords_df_global.empty:
                 _, _, missing_keywords = calculate_keyword_coverage(parsed_resume_content_global, keywords_df_global)
            
            prompt = f"""Rewrite the following resume bullet point to naturally incorporate relevant keywords from the list below, focusing on high-impact language and quantifiable results where possible. Maintain the original meaning and professional tone.
            
            Original Bullet Point:
            {bullet_text}
            
            Relevant Missing Keywords (try to include some):
            {', '.join(missing_keywords[:10]) if missing_keywords else 'None'}
            
            All Keywords (for context):
            {', '.join(keywords_df_global['keyword'].tolist()[:20]) if not keywords_df_global.empty else 'None'}
            
            Rewritten Bullet Point:"""
            
            try:
                safety_settings = [
                    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                ]
                model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings)
                response = model.generate_content(prompt)
                
                if not response.parts:
                     block_reason = "Unknown (empty response)"
                     if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                         block_reason = f"{response.prompt_feedback.block_reason_message}"
                     raise Exception(f"Blocked or empty response from API. Reason: {block_reason}")
                
                suggestion_text = response.text 
                
                clear_output(wait=True)
                display(HTML("<b>✨ AI Suggestion:</b>"))
                display(widgets.HTML(f"<pre style='white-space: pre-wrap; border: 1px solid #ccc; padding: 5px; background-color: #f9f9f9;'>{suggestion_text}</pre>"))
                
                apply_button = widgets.Button(
                    description="Apply Suggestion",
                    button_style='success',
                    tooltip='Replace the bullet point above with this suggestion',
                    icon='check'
                )
                apply_button.on_click(partial(on_apply_suggestion_click, bullet_widget, suggestion_text))
                apply_button_area.children = [apply_button]
                
            except Exception as e:
                clear_output(wait=True)
                error_message = str(e)
                display(HTML(f"<p style='color: red;'>❌ Error calling Google Gemini API: {error_message}</p>"))
        else:
            clear_output(wait=True)
            display(HTML("<p style='color: red;'>Error: Could not find the corresponding bullet point widget.</p>"))

# --- Helper Function to Build Interactive UI --- 
def build_interactive_ui(structured_data):
    global interactive_widgets, original_section_keys, structured_resume_global # Allow modification of global vars
    interactive_widgets = {} # Clear previous widgets
    structured_resume_global = structured_data # Store globally
    sections = []
    section_titles = []
    
    preferred_order = ['Header', 'Profile Summary', 'Technical Skills', 'Professional Experience', 'Projects Experience', 'Education', 'Certifications', 'Publications']
    section_keys = sorted(structured_data.keys(), key=lambda x: preferred_order.index(x) if x in preferred_order else len(preferred_order))
    original_section_keys = list(section_keys) # Store the initial order

    for section_name in section_keys:
        items = structured_data[section_name]
        section_content_widgets = [] # Use VBox for items within a section
        interactive_widgets[section_name] = []
        
        for item_index, item_text in enumerate(items):
            item_widget = widgets.Textarea(
                value=item_text,
                layout=widgets.Layout(width='90%', height='auto'), # Adjust width slightly
                description_tooltip=item_text,
                style={'description_width': 'initial'}
            )
            interactive_widgets[section_name].append(item_widget)
            
            item_row_widgets = [item_widget]
            if section_name in ['Professional Experience', 'Projects Experience']:
                suggest_button = widgets.Button(
                    description='Suggest', 
                    button_style='info', 
                    tooltip='Get AI suggestion for this bullet',
                    icon='lightbulb-o',
                    layout=widgets.Layout(width='auto')
                )
                suggest_button.on_click(partial(on_suggest_click, section_name, item_index))
                item_row_widgets.append(suggest_button)
            
            section_content_widgets.append(widgets.HBox(item_row_widgets))

        sections.append(widgets.VBox(section_content_widgets))
        section_titles.append(section_name)
        
    accordion = widgets.Accordion(children=sections)
    for i, title in enumerate(section_titles):
        accordion.set_title(i, title)
        
    section_order_input.value = ', '.join(section_keys)
    section_order_input.disabled = False
    # Enable save template widgets
    new_template_name_input.disabled = False
    save_template_button.disabled = False
        
    return [accordion] # Return as a list for VBox children

# --- Function to Refresh Template Dropdown --- 
def refresh_template_dropdown():
    """Reloads templates and updates the dropdown."""
    global job_types, job_types_templates
    current_job_type = job_type_dropdown.value
    current_template = template_dropdown.value
    
    job_types, job_types_templates = load_job_types_and_templates()
    
    job_type_dropdown.options = job_types
    if current_job_type in job_types:
        job_type_dropdown.value = current_job_type
    else:
        current_job_type = job_types[0] if job_types else None
        job_type_dropdown.value = current_job_type
        
    new_template_options = job_types_templates.get(current_job_type, [])
    template_dropdown.options = new_template_options
    if current_template in new_template_options:
        template_dropdown.value = current_template
    elif new_template_options:
        template_dropdown.value = new_template_options[0]
    else:
        template_dropdown.value = None

# Update template dropdown when job type changes
def update_templates_on_job_change(*args):
    refresh_template_dropdown() # Use the refresh function
    
job_type_dropdown.observe(update_templates_on_job_change, names='value')

# --- Save Template Handler --- 
def on_save_template_click(b):
    global interactive_widgets, job_types_templates
    with save_template_output_area:
        clear_output()
        template_name = new_template_name_input.value.strip()
        job_type = job_type_dropdown.value
        
        if not template_name:
            print("❌ Error: Please enter a name for the new template.")
            return
        if not job_type:
             print("❌ Error: Please select a job type first.")
             return
        if not interactive_widgets:
             print("❌ Error: No resume content loaded in the editor to save.")
             return
             
        sanitized_name = re.sub(r'[^a-zA-Z0-9_\-]+', '_', template_name).lower()
        if not sanitized_name:
             print("❌ Error: Invalid template name after sanitization.")
             return
             
        template_dir = os.path.join(job_types_base_dir, job_type, 'templates')
        template_path = os.path.join(template_dir, f"{sanitized_name}.md")
        
        current_order_str = section_order_input.value.strip()
        current_order = [s.strip() for s in current_order_str.split(',') if s.strip()]
        if set(current_order) != set(original_section_keys):
             print(f"❌ Error: Section order mismatch. Cannot save template.")
             return
             
        data_to_save = {}
        for section_name in current_order:
             if section_name in interactive_widgets:
                 data_to_save[section_name] = [widget.value for widget in interactive_widgets[section_name]]
             else:
                 data_to_save[section_name] = [] 
                 
        success = save_structure_as_template(data_to_save, template_path)
        
        if success:
            print(f"✅ Template '{sanitized_name}' saved for job type '{job_type}'.")
            new_template_name_input.value = '' 
            refresh_template_dropdown()
            if sanitized_name in template_dropdown.options:
                 template_dropdown.value = sanitized_name
        else:
            print(f"❌ Error saving template '{sanitized_name}'.")

save_template_button.on_click(on_save_template_click)

# Create main run button
run_button = widgets.Button(
    description='Generate Resume',
    button_style='success',
    icon='rocket'
)

output_area = widgets.Output()
# Widget for comparison output
comparison_output_area = widgets.Output()

# --- Main Run Button Handler --- 
def on_run_button_clicked(b):
    global interactive_widgets, parsed_resume_content_global, structured_resume_global, original_section_keys, keywords_df_global, original_coverage_results # Access global vars
    with output_area:
        clear_output()
        coverage_output.value = "" # Clear previous coverage info
        save_template_output_area.clear_output() # Clear save status
        comparison_output_area.clear_output() # Clear comparison status
        
        is_editing_flow = bool(interactive_widgets)
        
        if not is_editing_flow:
            # --- Initial Flow: Upload and Parse --- 
            interactive_area.children = [] 
            section_order_input.value = ''
            section_order_input.disabled = True
            new_template_name_input.disabled = True 
            save_template_button.disabled = True
            parsed_resume_content_global = None 
            structured_resume_global = None 
            original_section_keys = []
            original_coverage_results = {'percent': 0.0, 'found_kws': []} # Reset original coverage
            keywords_df_global = pd.DataFrame() 
            suggestion_output_area.clear_output() 
            apply_button_area.children = [] 
            uploaded_file_info = file_upload.value
            if not uploaded_file_info:
                print("❌ Error: Please upload your resume file.")
                return
                
            try:
                if isinstance(uploaded_file_info, list) and len(uploaded_file_info) > 0:
                    file_info = uploaded_file_info[0]
                elif isinstance(uploaded_file_info, dict) and len(uploaded_file_info) > 0:
                     file_info = list(uploaded_file_info.values())[0]
                else:
                     raise ValueError("Unexpected file upload format.")
                     
                uploaded_filename = file_info['name']
                uploaded_content = file_info['content']
                
                temp_resume_path = os.path.join(input_dir, uploaded_filename)
                with open(temp_resume_path, 'wb') as f:
                    f.write(uploaded_content)
                print(f"📄 Uploaded resume saved to: {temp_resume_path}")
                
            except Exception as e:
                print(f"❌ Error processing uploaded file: {e}")
                return
                
            parsed_resume_content = parse_resume(temp_resume_path)
            if parsed_resume_content is None:
                print(f"❌ Error: Failed to parse the uploaded resume file.")
                return
            else:
                print(f"✅ Successfully parsed resume content.")
                parsed_resume_content_global = parsed_resume_content 
                
            structured_resume = parse_structured_resume(temp_resume_path)
            if structured_resume:
                print(f"✅ Attempted to parse resume structure. Displaying editor.")
                interactive_area.children = build_interactive_ui(structured_resume)
            else:
                print(f"⚠️ Could not parse resume structure. Interactive editing might be limited.")
                interactive_widgets = {} 
                structured_resume_global = None 
        
        # --- Get Other User Inputs --- 
        basename = name_input.value.strip()
        if not basename:
            print("❌ Error: Please enter a name.")
            return
            
        job_type = job_type_dropdown.value
        if not job_type:
            print("❌ Error: Please select a job type.")
            return
            
        template = template_dropdown.value 
        if not template:
            print("❌ Error: Please select a template.")
            return
            
        do_upload = upload_dropdown.value
        
        # --- Load Keywords & Calculate Coverage --- 
        keywords_df_global = load_keywords_for_job(job_type, keywords_db_file)
        coverage_html = ""
        distribution_html = ""
        
        if keywords_df_global.empty:
             print(f"⚠️ Warning: No keywords loaded for job type '{job_type}'. Skipping keyword analysis.")
             coverage_html = "<i>Keyword analysis skipped (no keywords found for job type).</i>"
        elif parsed_resume_content_global is None:
             print(f"⚠️ Cannot calculate coverage, resume content not parsed yet.")
             coverage_html = "<i>Upload resume and click button first to see coverage.</i>"
        else:
             # Calculate and store original coverage if it's the first run
             if not is_editing_flow:
                 original_coverage_percent, original_found_kws, _ = calculate_keyword_coverage(parsed_resume_content_global, keywords_df_global)
                 original_coverage_results = {'percent': original_coverage_percent, 'found_kws': original_found_kws}
                 print(f"📊 Original Keyword Coverage: {original_coverage_percent:.2f}%")
                 coverage_html = f"<b>Original Keyword Coverage: {original_coverage_percent:.2f}%</b> ({len(original_found_kws)} / {len(keywords_df_global)} keywords found)<br>"
                 # Calculate missing based on original
                 missing_kws = list(set(keywords_df_global['keyword']) - set(original_found_kws))
             else:
                 # On second run (generation), just display original stored value
                 coverage_html = f"<b>Original Keyword Coverage: {original_coverage_results['percent']:.2f}%</b> ({len(original_coverage_results['found_kws'])} / {len(keywords_df_global)} keywords found)<br>"
                 # Calculate missing based on original
                 missing_kws = list(set(keywords_df_global['keyword']) - set(original_coverage_results['found_kws']))
                 
             # Display missing keywords (based on original)
             if missing_kws:
                 missing_kws_display = sorted(missing_kws, key=lambda kw: keywords_df_global[keywords_df_global['keyword'] == kw]['weight'].iloc[0], reverse=True)
                 coverage_html += f"<i>Missing Keywords (Top {min(10, len(missing_kws_display))} by weight):</i> {', '.join(missing_kws_display[:10])}"
             else:
                 coverage_html += "<i>All keywords found in original!</i>"
             
             # Distribution Analysis (based on original structure)
             if structured_resume_global:
                 distribution = analyze_keyword_distribution(structured_resume_global, keywords_df_global)
                 if distribution:
                     distribution_html = "<br><b>Keyword Distribution (Original):</b><ul style='margin-top: 0; padding-left: 20px;'>"
                     preferred_order_dist = ['Header', 'Profile Summary', 'Technical Skills', 'Professional Experience', 'Projects Experience', 'Education', 'Certifications', 'Publications']
                     dist_keys = sorted(distribution.keys(), key=lambda x: preferred_order_dist.index(x) if x in preferred_order_dist else len(preferred_order_dist))
                     for section in dist_keys:
                         count = distribution[section]['count']
                         distribution_html += f"<li>{section}: {count}</li>"
                     distribution_html += "</ul>"
                 else:
                      distribution_html = "<br><i>No keywords found in parsed sections.</i>"
             else:
                 distribution_html = "<br><i>Distribution analysis skipped (could not parse structure).</i>"
                 
        coverage_output.value = coverage_html + distribution_html 

        # --- Generation Flow (Only if is_editing_flow is True) --- 
        if is_editing_flow:
            print("🔄 Generating resume based on edited content...")
            suggestion_output_area.clear_output() 
            apply_button_area.children = [] 
            
            # --- Validate and Get Section Order --- 
            desired_order_str = section_order_input.value.strip()
            desired_order = [s.strip() for s in desired_order_str.split(',') if s.strip()]
            
            if set(desired_order) != set(original_section_keys):
                print(f"❌ Error: Section order mismatch. Ensure all original sections ({', '.join(original_section_keys)}) are included exactly once in the order list.")
                section_order_input.value = ', '.join(original_section_keys)
                return
            else:
                print(f"📑 Using section order: {', '.join(desired_order)}")
            
            # Retrieve edited data from widgets in the *desired* order
            edited_data = {}
            for section_name in desired_order:
                 if section_name in interactive_widgets:
                     edited_data[section_name] = [widget.value for widget in interactive_widgets[section_name]]
                 else:
                     print(f"Warning: Section '{section_name}' from order list not found in widgets.")
                     edited_data[section_name] = []
            
            # --- Setup Progress Bar --- 
            total_steps = 1 # Generation step
            if do_upload != 'skip':
                total_steps += 1 # PDF/Upload step
            
            progress = tqdm.tqdm(total=total_steps, desc="Resume Generator")
            
            # --- Generate Resume --- 
            basename_underscore = basename.replace(' ', '_').lower()
            base_output = os.path.join(output_dir, f"{basename_underscore}_resume.md")
            generated_content = generate_resume_from_structure(edited_data, base_output, keywords_df=keywords_df_global)
            
            if generated_content is None:
                print("❌ Error: Resume generation failed.")
                progress.close()
                return
                
            progress.update(1)
            
            # --- Comparison Calculation --- 
            comparison_html = ""
            if parsed_resume_content_global is not None and not keywords_df_global.empty:
                final_coverage_percent, final_found_kws, _ = calculate_keyword_coverage(generated_content, keywords_df_global)
                original_percent = original_coverage_results['percent']
                original_found = original_coverage_results['found_kws']
                improvement = final_coverage_percent - original_percent
                added_keywords = sorted(list(set(final_found_kws) - set(original_found)))
                
                comparison_html = f"<hr><b>Comparison:</b><br>"
                comparison_html += f"Original Coverage: {original_percent:.2f}%<br>"
                comparison_html += f"Final Coverage: {final_coverage_percent:.2f}% "
                if improvement > 0.01:
                    comparison_html += f"<span style='color: green;'>(+{improvement:.2f}%)</span><br>"
                elif improvement < -0.01:
                     comparison_html += f"<span style='color: red;'>({improvement:.2f}%)</span><br>"
                else:
                     comparison_html += "(No Change)<br>"
                     
                if added_keywords:
                    comparison_html += f"Keywords Added/Improved: {', '.join(added_keywords)}"
                else:
                    comparison_html += "No new keywords added."
            
            # Display comparison results
            with comparison_output_area:
                 display(HTML(comparison_html))
            
            # --- Generate PDF & Upload --- 
            upload_message = ""
            if do_upload != 'skip':
                pdf_file = generate_pdf(base_output, output_dir)
                if pdf_file:
                    if do_upload is True:
                        drive_link = upload_to_drive(pdf_file, drive_folder_id)
                        upload_message = f"and uploaded to Google Drive ({drive_link})" if drive_link else "and attempted upload to Google Drive"
                    else:
                        upload_message = f"(PDF generated, not uploaded to Google Drive)"
                    progress.update(1)
                else:
                    upload_message = f"but PDF generation failed"
            else:
                upload_message = "(PDF generation skipped)"
            
            progress.close()
            print(f"✅ Done! Your resume has been generated {upload_message}.")
            print(f"📄 Output files are in: {output_dir}")
            # Clear interactive state after successful generation
            interactive_widgets = {}
            parsed_resume_content_global = None
            structured_resume_global = None
            original_section_keys = []
            original_coverage_results = {'percent': 0.0, 'found_kws': []}
            keywords_df_global = pd.DataFrame()
            interactive_area.children = []
            section_order_input.value = ''
            section_order_input.disabled = True
            new_template_name_input.value = '' 
            new_template_name_input.disabled = True
            save_template_button.disabled = True
            suggestion_output_area.clear_output()
            apply_button_area.children = []
        else:
             # This is the first click (parse & display editor)
             print("📄 Review the editor above. You can edit content and section order. Click 'Generate Resume' again when ready.")

# Connect the main run button
run_button.on_click(on_run_button_clicked)

# --- Display Widgets --- 
display(widgets.HTML("<h3>Resume Generator Options</h3>"))
display(file_upload)
display(name_input)
display(job_type_dropdown)
display(template_dropdown)
display(upload_dropdown)
display(widgets.HTML("<h4>Keyword Analysis:</h4>")) # Renamed header
display(coverage_output) # Displays coverage and distribution
display(widgets.HTML("<h4>Interactive Resume Editor:</h4>"))
display(interactive_area)
display(section_order_input) # Display section order input
display(widgets.HTML("<h5>Save Current Edits as New Template:</h5>")) # Header for saving
display(widgets.HBox([new_template_name_input, save_template_button])) # Display save widgets
display(save_template_output_area) # Area for save status
display(widgets.HTML("<h5>AI Suggestions:</h5>")) # Header for suggestions
display(suggestion_output_area) # Area for suggestion output
display(apply_button_area) # Area for the apply button
display(run_button)
display(output_area)
display(comparison_output_area) # Area for comparison output